In [ ]:
%pylab inline
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import numpy as np

In [ ]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms

In [ ]:
transform = transforms.Compose([
                                transforms.CenterCrop(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5], [0.5])
                               ])